## 네이버 뉴스 검색 및 크롤링
- 코로나 검색해서 첫 페이지의 모든 기사의 제목과 언론사, 요약 크롤링
- https://search.naver.com/search.naver?where=news&sm=tab_jum&query=


In [1]:
def get_newslist(driver):
    news_titles = []
    news_presses = []
    news_summaries = [] 
    news_list = driver.find_elements(by=By.CLASS_NAME, value='news_wrap.api_ani_send')
    
    for news in news_list:
        #기사 제목 추출
        news_title = news.find_element(by=By.CLASS_NAME, value='news_tit')
        news_titles.append(news_title.get_attribute('title'))
        #언론사 추출
        news_press = news.find_element(by=By.CSS_SELECTOR, value='div.news_info > div.info_group > a.info.press')
        news_presses.append(news_press.text.replace("언론사 선정", ''))
        #기사 요약 추출
        news_summary = news.find_element(by=By.CLASS_NAME, value='dsc_wrap')
        news_summaries.append(news_summary.text)
        
    return [news_titles, news_presses, news_summaries]


def save_table(data, index, title):
    #데이터프레임형식으로 변환 및 저장
    import pandas as pd
    news_table = pd.DataFrame(data, index=index).T
    news_table.to_csv(title)


def selenium_service(url='https://search.naver.com/search.naver?where=news&sm=tab_jum&query=', TIMEOUT=3, search_key="none", id_value='nx_query'):
    try: 
        service = Service(executable_path=ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        driver.get(url)
        driver.implicitly_wait(TIMEOUT)
        print("페이지 접속합니다.")
        
        search = driver.find_element(by=By.ID, value=id_value)
        search.click()
        search.send_keys(search_key)
        search.send_keys(Keys.RETURN)
        print(f'{search_key} 검색 후 접속')
        driver.implicitly_wait(TIMEOUT)
        
        get = get_newslist(driver)
        return get 

    except Exception:
        raise
    
    finally:
        if driver is not None:
            driver.quit()

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

data = selenium_service(search_key='코로나')
save_table(data, ['제목', '언론사', '기사 요약'], "네이버 코로나 기사 리스트.csv")

페이지 접속합니다.
코로나 검색 후 접속


## 누적관객수 추출
- 누적관객수 1000만 이상인 영화명, 누적매출액, 개봉일 출력
- http://www.kobis.or.kr/kobisopenapi/homepg/apiservice/searchServiceInfo.do
- http://www.kobis.or.kr/kobisopenapi/homepg/board/findTutorial.do?targetId=section_3
 

In [72]:
def get_DailyBoxOfficeList(targetDt=str):
    endpoint = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.'
    contentType="json"
    key = '6940dfa30d685d37fce05cbacd26f7cc'
    
    #보고자 하는 해당 날짜의 박스오피스 리스트 추출
    paramset = contentType+'?key='+key+'&targetDt='+targetDt 
    url = endpoint+paramset
    result = requests.get(url)
    movie_data = result.json()
    return movie_data

def get_audiAcc(movie_data, condition):
    movie_list = movie_data['boxOfficeResult']['dailyBoxOfficeList']
    for movie in movie_list:
        if movie['audiAcc'] != 'null':
            if int(movie['audiAcc']) >= condition:
                print(movie['movieNm'], movie['salesAcc'], movie['openDt'])


In [79]:
import requests

movie_list = get_DailyBoxOfficeList('20220924')
get_audiAcc(movie_list, 10000000)
